# APS 4 - PROJETO DE SOFTWARE PARA ANÁLISE DE TRELIÇAS PLANAS

In [20]:
from funcoesTermosol import geraSaida, importa, plota
import numpy as np
from math import sqrt 


In [50]:
# numero de nos [nn]
# matriz dos nos [N]
# numero de membros [nm]
# matriz de incidencia [Inc]
# numero de cargas [nc]
# vetor carregamento [F]
# numero de restricoes [nr]
# vetor de restricoes [R]


matriz_um = np.array([[1, -1], [-1, 1]])


class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y

        self.rx = None
        self.ry = None


class Elemento:
    def __init__(self, no1, no2, E, A):
        self.E = E
        self.A = A
        self.l = sqrt((no1.x - no2.x)**2 + (no1.y - no2.y)**2)
        self.cos = (no2.x - no1.x)/self.l
        self.sin = (no2.y - no1.y)/self.l
        self.T = np.array([[self.cos, self.sin, 0, 0],
                          [0, 0, self.cos, self.sin]])
        self.T_transposta = np.transpose(self.T)

        self.ke = np.dot(np.dot(self.T_transposta *
                         (self.E*self.A/self.l), matriz_um), self.T)


[nn, N, nm, Inc, nc, F, nr, R] = importa('entrada.xls')

Px = F[4][0]
Py = F[5][0]
sigma_tracao = 1570e6
sigma_compressao = sigma_tracao

# criar um objeto Node para cada nó
node1 = Node(N[0, 0], N[1, 0])
node2 = Node(N[0, 1], N[1, 1])
node3 = Node(N[0, 2], N[1, 2])
# Node1 tem restrição em x
# Node2 tem restrição em x e y

Elemento12 = Elemento(node1, node2, Inc[0, 2], Inc[0, 3])
Elemento23 = Elemento(node2, node3, Inc[1, 2], Inc[1, 3])
Elemento31 = Elemento(node3, node1, Inc[2, 2], Inc[2, 3])

# Create a global stiffness matrix superpositioning the local stiffness matrices
K = np.zeros((2*nn, 2*nn))

AUXS = np.array([[1, 2, 3, 4], [3, 4, 5, 6], [5, 6, 1, 2]])
ELEMENTOS = [Elemento12, Elemento23, Elemento31]
for aux, elemento in zip(AUXS,ELEMENTOS):
    for i in range(len(aux)):
        for j in range(len(aux)):
            K[aux[i]-1, aux[j]-1] += elemento.ke[i, j]

# Vetor global de forças concentradas (Aula 25 -> Slide 22)
PG = np.array([node1.rx, 0, node2.rx, node2.ry, Px, Py], dtype=object)

# Aplicando condição de contorno (Aula 25 -> Slide 23)
# Removendo linhas e colunas que correspondem aos graus de liberdade das reações dos nós (1, 3, 4)
# Documentação para o np.delete: https://note.nkmk.me/en/python-numpy-delete/#:~:text=Using%20the%20NumPy%20function%20np,from%20the%20NumPy%20array%20ndarray%20.&text=Specify%20the%20axis%20(dimension)%20and,a%20slice%20or%20a%20list.
K_CC = np.delete(np.delete(K, [0,2,3], 0), [0,2,3], 1)
PG_CC = np.delete(PG, [0,2,3], 0)

# Calcular os deslocamentos nodais (Aula 25 -> Slide 24)

# Calcular reações de apoio (Aula 25 -> Slide 25)

# Calcular deformação e tensão em cada elemento (Aula 25 -> Slide 8)





[None 0 None None 150.0 -100.0]


[0 150.0 -100.0]
